In [19]:
liked_books = ["3850639", "12961964", "27362503"] #book ids of books I like

In [20]:
!head book_id_map.csv #interactions+ratings

book_id_csv,book_id
0,34684622
1,34536488
2,34017076
3,71730
4,30422361
5,33503613
6,33517540
7,34467031
8,6383669


In [29]:
csv_book_mapping = {}

with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id
#bc book ids are different in each data set, must map

In [31]:
csv_book_mapping['0']

'34684622'

In [23]:
!wc -l goodreads_interactions.csv

 228648343 goodreads_interactions.csv


In [24]:
!ls -lh  | grep goodreads_interactions

-rw-r--r--   1 shaimahussaini  staff   4.0G Jun 30 23:35 goodreads_interactions.csv


In [25]:
!head goodreads_interactions.csv #file that has how each user rated each book

user_id,book_id,is_read,rating,is_reviewed
0,948,1,5,0
0,947,1,5,1
0,946,1,5,0
0,945,1,5,0
0,944,1,5,0
0,943,1,5,0
0,942,1,5,0
0,941,1,5,0
0,940,1,5,0


In [32]:
overlap_users = set(); #every element is unique
#will contain any user who read same book as us and rated book highly



    
with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",") #underscore means don't care about that field, convention
        
        if user_id in overlap_users:
            continue

        try:
            rating = int(rating)
        except ValueError:
            continue
        
        book_id = csv_book_mapping[csv_id] #use book id to make list of liked books
        
        if book_id in liked_books and rating >= 4:
                overlap_users.add(user_id)

In [34]:
rec_lines = [] #potential recs

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating])

In [35]:
len(overlap_users)

13648

In [36]:
len(rec_lines) #a lot of books

9030218

In [45]:
import pandas as pd

recs = pd.DataFrame(rec_lines, columns=["user_id", "book_id", "rating"])
recs["book_id"] = recs["book_id"].astype(str)

In [46]:
recs

,user_id,book_id,rating
0,0,12,5
1,0,21,5
2,0,30,5
3,0,45,5
4,0,1,5
...,...,...,...
9030213,875876,8520610,0
9030214,875876,9013,0
9030215,875876,29751398,0
9030216,875876,29780253,5


In [41]:
top_recs = recs["book_id"].value_counts().head(10) #counts how many times each book id occurs and show most frequent ones

In [47]:
top_recs = top_recs.index.values

In [48]:
#left with only book id and value count
#now need to get from book id to title
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [49]:
books_titles.head()

,book_id,title,ratings,url,cover_image,mod_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,the devils notebook


In [50]:
books_titles[books_titles["book_id"].isin(top_recs #top 10 liked by people who liked our liked

,book_id,title,ratings,url,cover_image,mod_title
73489,7260188,"Mockingjay (The Hunger Games, #3)",1743362,https://www.goodreads.com/book/show/7260188-mo...,https://images.gr-assets.com/books/1358275419m...,mockingjay the hunger games 3
168466,22609310,Confess,73455,https://www.goodreads.com/book/show/22609310-c...,https://images.gr-assets.com/books/1412280173m...,confess
284473,2767052,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
323031,41865,"Twilight (Twilight, #1)",3941381,https://www.goodreads.com/book/show/41865.Twil...,https://images.gr-assets.com/books/1361039443m...,twilight twilight 1
535617,17788401,Ugly Love,111027,https://www.goodreads.com/book/show/17788401-u...,https://images.gr-assets.com/books/1399179135m...,ugly love
611301,27362503,It Ends with Us,88278,https://www.goodreads.com/book/show/27362503-i...,https://images.gr-assets.com/books/1470427482m...,it ends with us
833311,6148028,"Catching Fire (The Hunger Games, #2)",1854746,https://www.goodreads.com/book/show/6148028-ca...,https://images.gr-assets.com/books/1358273780m...,catching fire the hunger games 2
878545,3,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
982393,15717943,"Hopeless (Hopeless, #1)",194321,https://www.goodreads.com/book/show/15717943-h...,https://images.gr-assets.com/books/1353489892m...,hopeless hopeless 1
1095301,11870085,The Fault in Our Stars,2429317,https://www.goodreads.com/book/show/11870085-t...,https://images.gr-assets.com/books/1360206420m...,the fault in our stars


In [51]:
#need to differentiate people like us from generic liked books
all_recs = recs["book_id"].value_counts()

In [52]:
all_recs

27362503    10631
2767052      8507
11870085     8044
41865        7658
17788401     7047
            ...  
13288551        1
6277561         1
2722251         1
35499148        1
1653086         1
Name: book_id, Length: 704480, dtype: int64

In [53]:
all_recs = all_recs.to_frame().reset_index()

In [55]:
all_recs.columns = ["book_id", "book_count"]

In [56]:
all_recs

,book_id,book_count
0,27362503,10631
1,2767052,8507
2,11870085,8044
3,41865,7658
4,17788401,7047
...,...,...
704475,13288551,1
704476,6277561,1
704477,2722251,1
704478,35499148,1


In [57]:
all_recs = all_recs.merge(books_titles, how="inner", on="book_id") #if data doesnt exist in both, then get rid of it

In [58]:
all_recs

,book_id,book_count,title,ratings,url,cover_image,mod_title
0,27362503,10631,It Ends with Us,88278,https://www.goodreads.com/book/show/27362503-i...,https://images.gr-assets.com/books/1470427482m...,it ends with us
1,2767052,8507,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
2,11870085,8044,The Fault in Our Stars,2429317,https://www.goodreads.com/book/show/11870085-t...,https://images.gr-assets.com/books/1360206420m...,the fault in our stars
3,41865,7658,"Twilight (Twilight, #1)",3941381,https://www.goodreads.com/book/show/41865.Twil...,https://images.gr-assets.com/books/1361039443m...,twilight twilight 1
4,17788401,7047,Ugly Love,111027,https://www.goodreads.com/book/show/17788401-u...,https://images.gr-assets.com/books/1399179135m...,ugly love
...,...,...,...,...,...,...,...
616753,11493956,1,The Scioneer,24,https://www.goodreads.com/book/show/11493956-t...,https://s.gr-assets.com/assets/nophoto/book/11...,the scioneer
616754,6277561,1,Book Of Werewolves,213,https://www.goodreads.com/book/show/6277561-bo...,https://s.gr-assets.com/assets/nophoto/book/11...,book of werewolves
616755,2722251,1,All About Colour,19,https://www.goodreads.com/book/show/2722251-al...,https://images.gr-assets.com/books/1320427504m...,all about colour
616756,35499148,1,Simon Fayter and the Doors of Bone (Simon Fayt...,21,https://www.goodreads.com/book/show/35499148-s...,https://images.gr-assets.com/books/1498167300m...,simon fayter and the doors of bone simon fayter 1


In [59]:
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"]) / all_recs["ratings"] #penalized based on v popular book, more ratings

In [60]:
all_recs.sort_values("score", ascending=False).head(10)

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
3230,31681941,394,The Comfort Zone,36,https://www.goodreads.com/book/show/31681941-t...,https://s.gr-assets.com/assets/nophoto/book/11...,the comfort zone,4312.111111
2071,24909347,564,"Obsidio (The Illuminae Files, #3)",82,https://www.goodreads.com/book/show/24909347-o...,https://images.gr-assets.com/books/1501704611m...,obsidio the illuminae files 3,3879.219512
5439,32333338,253,Save the Date,19,https://www.goodreads.com/book/show/32333338-s...,https://images.gr-assets.com/books/1510611507m...,save the date,3368.894737
3715,34705684,353,"Cracked Kingdom (The Royals, #5)",39,https://www.goodreads.com/book/show/34705684-c...,https://s.gr-assets.com/assets/nophoto/book/11...,cracked kingdom the royals 5,3195.102564
2584,29749098,476,"Catwoman: Soulstealer (DC Icons, #3)",73,https://www.goodreads.com/book/show/29749098-c...,https://s.gr-assets.com/assets/nophoto/book/11...,catwoman soulstealer dc icons 3,3103.780822
1273,31050237,812,"Untitled (A Court of Thorns and Roses, #4)",264,https://www.goodreads.com/book/show/31050237-u...,https://s.gr-assets.com/assets/nophoto/book/11...,untitled a court of thorns and roses 4,2497.515152
2176,30809786,544,"A Reaper at the Gates (An Ember in the Ashes, #3)",124,https://www.goodreads.com/book/show/30809786-a...,https://images.gr-assets.com/books/1507476834m...,a reaper at the gates an ember in the ashes 3,2386.580645
1992,31050358,581,"Untitled (A Court of Thorns and Roses, #5)",146,https://www.goodreads.com/book/show/31050358-u...,https://s.gr-assets.com/assets/nophoto/book/11...,untitled a court of thorns and roses 5,2312.061644
3937,26181560,335,"Forget You, Ethan",49,https://www.goodreads.com/book/show/26181560-f...,https://images.gr-assets.com/books/1504787872m...,forget you ethan,2290.306122
5285,35133826,259,Say You'll Remember Me,31,https://www.goodreads.com/book/show/35133826-s...,https://images.gr-assets.com/books/1500739206m...,say youll remember me,2163.903226


In [66]:
popular_recs = all_recs[all_recs["book_count"] > 200].sort_values("score", ascending=False)

In [65]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)


popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'url': make_clickable, 'cover_image': show_image})


,book_id,book_count,title,ratings,url,cover_image,mod_title,score
3230,31681941,394,The Comfort Zone,36,Goodreads,,the comfort zone,4312.111111
2071,24909347,564,"Obsidio (The Illuminae Files, #3)",82,Goodreads,,obsidio the illuminae files 3,3879.219512
5439,32333338,253,Save the Date,19,Goodreads,,save the date,3368.894737
3715,34705684,353,"Cracked Kingdom (The Royals, #5)",39,Goodreads,,cracked kingdom the royals 5,3195.102564
2584,29749098,476,"Catwoman: Soulstealer (DC Icons, #3)",73,Goodreads,,catwoman soulstealer dc icons 3,3103.780822
1273,31050237,812,"Untitled (A Court of Thorns and Roses, #4)",264,Goodreads,,untitled a court of thorns and roses 4,2497.515152
2176,30809786,544,"A Reaper at the Gates (An Ember in the Ashes, #3)",124,Goodreads,,a reaper at the gates an ember in the ashes 3,2386.580645
1992,31050358,581,"Untitled (A Court of Thorns and Roses, #5)",146,Goodreads,,untitled a court of thorns and roses 5,2312.061644
3937,26181560,335,"Forget You, Ethan",49,Goodreads,,forget you ethan,2290.306122
5285,35133826,259,Say You'll Remember Me,31,Goodreads,,say youll remember me,2163.903226


In [ ]:
#first we built book search engine to find book_ids, created matrix using vectorizer, processing
#created list of books that we liked to find who liked same books as me and what books did they like